In [1]:
import numpy as np

Testing equations:
1. $\mathbb{E}_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)+O(p^2)$
2. $\mathbb{E}_\xi\Delta_L=-p\nabla_x y_L(x)+O(p^2x_i^2)$
3. $Var_\xi\Delta_L=p\sum\left(y(\vec{x}-x_i)-y(\vec{x})\right)^2+O(p^2)$
4. $Var_\xi\Delta_L=p(\nabla_x y_L(x))^2x^2+O(p^2x_i^2)$
5. $Var_\xi\Delta_L=pW^2_LK^2_{L-1}\odot W^2_{L-1}...W_2^2K_1^2\odot W_1^2x^2+O(p^2x_i^2w^2)$

Testing theorems:
1. $\mathbb{E}_{x}\mathbb{E}_{\xi}\Delta_L=O(p^2x_i^2)$
2. 	$\sum\limits_{i,j}\frac{\partial y_k}{\partial w_{ij}}w_{ij}=\sum\limits_{j}\frac{\partial y_k}{\partial x_j}x_j
    = -\frac{1}{p}(\mathbb{E}_\xi\Delta_L^k+O(p^2))$